## Validação dos dados EOG

Neste notebook está incluído os seguintes passos:
- Aplicação de características;
- Criação do vetor de características;
- Normalização de dados;
- Seleção de características;
- Classificação dos dados.

Uma característica é uma propriedade individual mensurável ou característica de um fenômeno que está sendo observado. Em nosso caso de EOG, uma característica pode ser extraída no domínio do tempo ou no domínio da frequência. As características a seguir foram retiradas do artigo *EMG Feature Extraction for Tolerance of White Gaussian Noise* \[1\].

#### Domínio do tempo

1. Willison Amplitude (WAMP)

    > $ \sum_{i=1}^{N-1}f(|x_i - x_{i+1}|) $
    
    > $ f(x) = \begin{cases} 1 & \text{if } x \gt threshold \\ 0 & \text{otherwise} \end{cases} $

2. Variance of EMG (VAR)

    > $ \frac{1}{N-1}\sum_{i=1}^{N}x_i^2 $

3. Root Mean Square (RMS)

    > $ \sqrt{\frac{1}{N}\sum_{i=1}^{N}|x_i|^2} $

4. Waveform Length (WL)
    
    > $ \sum_{i=1}^{N-1}|x_{i+1} - x_i| $

5. Zero Crossing (ZC)

    > $ \sum_{i=1}^{N}sgn(x_i) $
    
    > $ sgn(x) = \begin{cases} 1 & \text{if } x_i * x_{i+1} \leq 0 \\ 0 & \text{otherwise} \end{cases} $

#### Domínio da frequência

1. Median Frequency (FMD)

    > $ \frac{1}{2}\sum_{j=1}^{M}PSD_j $

2. Mean Frequency (FMN)

    > $\sum_{j=1}^{M}f_j PSD_j / \sum_{j=1}^{M}PSD_j$
    
    > $ f_j = j * SampleRate / 2 * M $

3. Modified Median Frequency (MMDF)

    > $ \frac{1}{2}\sum_{j=1}^{M}A_j $
    
    > $ A_j = Amplitude\ do\ espectro\ j $

4. Modified Frequency Mean (MMNF)

    > $ \sum_{j=1}^{M}f_jAj / \sum_{j=1}^{M}Aj $


\[1\] Phinyomark, Angkoon & Limsakul, Chusak & Phukpattaranont, P.. (2008). EMG Feature Extraction for Tolerance of White Gaussian Noise.
[Disponível neste link](https://www.researchgate.net/publication/263765853_EMG_Feature_Extraction_for_Tolerance_of_White_Gaussian_Noise)

**Tarefa 1**: Descrever as características de acordo com o artigo citado e outros disponíveis relacionados. O que está querendo "ser visto" em cada característica? Qual é o significado matemático de cada uma delas?

#### Aplicando as características

É necessário implementar as características, geralmente em formato de funções ou métodos, para que seja possível aplicar tais funções aos dados de entrada e obter as características resultantes. A seguir temos a implementação das características `VAR` & `RMS` (domínio do tempo) e `FDM` & `MMDF` (domínio da frequência).

In [58]:
from math import prod
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder

# funções auxiliares
def PSD(w):
    ''' definição da função PSD para o sinal no domínio da frequência '''
    return np.abs(w) ** 2


# funções de extração de características

def var(x):
    return np.sum(x ** 2, axis=-1) / (np.prod(x.shape) - 1)

def rms(x):
    return np.sqrt(np.sum(np.abs(x) ** 2, axis=-1) / (np.prod(x.shape) - 1))

def fmd(w):
    return np.sum(PSD(w), axis=-1) / 2

def mmdf(w):
    return np.sum(np.abs(w), axis=-1) / 2

**Tarefa 2**: Implemente todas as características apresentadas neste tutorial em formato de funções. Sinta-se livre também para buscar e implementar características além das apresentadas, citando as fontes de tais características.


#### Vetor de características

Ao final da implementação e seleção das características, deve ser escolhida as características e então teremos um vetor com todas elas implementadas.

O vetor de características estará organizado da seguinte forma (exemplo p/ VAR, RMS, RDM e MMDF):

| ID sample | VAR1 | RMS1 | FMD1 | MMDF1 | VAR2 | RMS2 | FMD2 | MMDF2 | Classe |
|:---------:|:----:|:----:|:----:|:-----:|------|------|------|-------|:------:|
|     1     |  v1  |  v1  |  v1  |   v1  | v1   | v1   | v1   | v1    |    0   |
|     2     |  v2  |  v2  |  v2  |   v2  | v2   | v2   | v2   | v2    |    0   |
|    ...    |  ... |  ... |  ... |  ...  | ...  | ...  | ...  | ...   |   ...  |
|     N     |  vN  |  vN  |  vN  |   vN  | vN   | vN   | vN   | vN    |    7   |

#### Implementação do vetor

In [59]:
# carregando dados do "prepare.ipynb"

x_all = np.load("./datasets/all_time.npy")
w_all = np.load("./datasets/all_freq.npy")

x = np.load("./datasets/gabi_time.npy")
w = np.load("./datasets/gabi_freq.npy")

print("Shape dos vetores orginais:", x.shape, w.shape)

# aplicando características

data_var = var(x)
data_rms = rms(x)

data_fmd = fmd(w)
data_mmdf = mmdf(w)

data_var.shape, data_rms.shape, data_fmd.shape, data_mmdf.shape


Shape dos vetores orginais: (2, 33, 64) (2, 33, 33)


((2, 33), (2, 33), (2, 33), (2, 33))

### Todos os individuos

In [66]:
# União do vetor de características inicial
features_all = np.array([var(x_all), rms(x_all), fmd(w_all), mmdf(w_all)])

print("features_all", features_all.shape)

# organização das dimensões
features_all = features_all.transpose(1, 3, 0, 2)
print("features_all", features_all.shape)

# criar vetor de características definitivo
features_all = features_all.reshape(features_all.shape[0] * features_all.shape[1],
                            features_all.shape[2] * features_all.shape[3])

features_all.shape

features_all (4, 56, 2, 33)
features_all (56, 33, 4, 2)


(1848, 8)

### Um individuo

In [74]:


# 1 INDIVIDUO

# União do vetor de características inicial
features = np.array([data_var, data_rms, data_fmd, data_mmdf])
print(features.shape)

# organização das dimensões
features = features.transpose(2, 0, 1)
print(features.shape)

# # criar vetor de características definitivo
features = features.reshape(features.shape[0], features.shape[1] * features.shape[2])

features.shape

(4, 2, 33)
(33, 4, 2)


(33, 8)

*Tarefa 3*: Realização da normalização dos dados utilizando ferramentas já conhecidas

In [62]:
# # aplicando normalização
# X = StandardScaler().fit_transform(features_all)

# # código

*Tarefa 4*: Realização da seleção de características, utilizando ferramentas já conhecidas

Criação do vetor de *labels*

In [63]:
labels_str = ['dir', 'esq', 'cima', 'baixo', 'cima', 'baixo',
'baixo', 'esq', 'dir', 'baixo', 'dir', 'dir', 'esq', 'cima',
'baixo', 'cima', 'esq', 'dir', 'cima', 'esq', 'baixo', 'esq',
'dir', 'esq', 'cima', 'dir', 'cima', 'baixo']

# transformando para numérico
lab_dict = {'dir': 0, 'esq': 1, 'cima': 2, 'baixo': 3}
labels_num = [lab_dict[item] for item in labels_str]
print('labels_num', len(labels_num))

# criação do vetor de labels final
y = np.array(labels_num * int(features.shape[0] / len(labels_num)))
len(y)

labels_num 28


56

In [64]:
# aplicando seleção de características
from sklearn.feature_selection import SelectKBest, chi2, f_regression, f_classif

num_of_chars = features.shape[1]
ks = [2**i for i in range(0, 12)]
initial_k = num_of_chars
accs = []

aux_features = np.abs(features)
print(aux_features.shape, y.shape)
for k in ks:
    X_new = SelectKBest(f_classif, k=initial_k - k).fit_transform(aux_features, y)
    print(y, X_new.shape)
# código    

(66, 4) (56,)


ValueError: Found input variables with inconsistent numbers of samples: [66, 56]

*Tarefa 5*: Realização da classificação utilizando `SVM`.

In [ ]:
# aplicando a classificação

# código